## File for creating downsampled daily counts of EQs and their related parameters 

In [5]:
import pandas as pd
import dask.dataframe as dd
import hvplot.pandas  # noqa
import hvplot.dask  # noqa
import numpy as np
from scipy import stats

In [6]:
!ls /home/jovyan/data/botpt/axial_seismic/

hypo71_11_2019.txt  hypo71_2018.dat


In [7]:
!head /home/jovyan/data/botpt/axial_seismic/hypo71_11_2019.txt

yyyymmdd HHMM SSS.SS Lat(D M)  Lon(D M)  Depth    MW NWR GAP DMIN  RMS  ERH  ERZ            ID         PMom    SMom
20150122 0000 27.54 45 56.81 129 59.92   0.55  -0.31  7 206  1.2 0.03  0.8  0.4         20830      3.8e+17 3.9e+17
20150122 0008 58.89 45 56.96 129 59.70   0.00   0.02 10 149  1.2 0.09  0.2  0.2         20831      1.9e+18 7.8e+17
20150122 0202 41.04 45 56.47 130  0.89   0.67  -0.03 12 200  0.8 0.03  0.9  0.7         20836      1.4e+18 7.2e+17
20150122 0219 56.90 45 54.93 129 57.59   1.60   0.26  7 325  1.8 0.06  1.4  2.5         20837      3.3e+18 2.2e+18
20150122 0236  7.08 45 56.80 129 59.71   0.51  -0.15  8 160  1.2 0.06  0.5  0.2         20840      6.7e+17 6.8e+17
20150122 0239 20.66 45 56.93 129 59.48   1.56   0.07 12 148  0.9 0.08  0.5  1.3         20841      1.3e+18 1.6e+18
20150122 0242 34.06 45 56.70 129 59.34   1.09   0.27 12 108  0.9 0.05  0.3  1.4         20843      2.7e+18 3.1e+18
20150122 0250 15.94 45 57.96 129 59.53   1.45   0.54 14 286  1.8 0.03  0.7  1.1

In [8]:
seismic_file = '/home/jovyan/data/botpt/axial_seismic/hypo71_11_2019.txt'
df_seismic = pd.read_csv(seismic_file, delim_whitespace=True, dtype=object)
df_seismic['datetime'] = df_seismic['yyyymmdd'] + 'T' + \
            df_seismic['HHMM'].str.slice(start=0, stop=2) + ':' + \
            df_seismic['HHMM'].str.slice(start=2) 
df_seismic.index = pd.to_datetime(df_seismic['datetime'].values)
df_seismic['datetime'] = pd.to_datetime(df_seismic['datetime'].values)
df_seismic = df_seismic.loc['2015-05-01 00:00:00':'2019-10-14 00:00:00']
df_seismic.datetime.astype(np.int64).values/1e64
df_seismic['date'] =pd.DatetimeIndex(df_seismic.datetime).date
df_seismic['Depth']= df_seismic['Depth'].astype(np.float).values
df_seismic['MW']= df_seismic['MW'].astype(np.float).values
df_seismic['RMS']= df_seismic['RMS'].astype(np.float).values
del df_seismic['NWR']
del df_seismic['GAP']
del df_seismic['DMIN']
del df_seismic['ERH']
del df_seismic['ERZ']
del df_seismic['ID']
del df_seismic['PMom']
del df_seismic['SMom']
del df_seismic['yyyymmdd']
del df_seismic['HHMM']
del df_seismic['SSS.SS']
del df_seismic['Lat(D']
del df_seismic['M)']
del df_seismic['Lon(D']
del df_seismic['M).1']
df_seismic.head()

,Depth,MW,RMS,datetime,date
2015-05-01 00:02:00,1.91,0.48,0.03,2015-05-01 00:02:00,2015-05-01
2015-05-01 00:27:00,0.82,0.03,0.01,2015-05-01 00:27:00,2015-05-01
2015-05-01 00:28:00,0.43,-0.23,0.01,2015-05-01 00:28:00,2015-05-01
2015-05-01 00:44:00,0.71,-0.16,0.01,2015-05-01 00:44:00,2015-05-01
2015-05-01 00:46:00,0.44,-0.34,0.04,2015-05-01 00:46:00,2015-05-01


## Create new dataframe with count per day

In [12]:
df_eqMean=df_seismic.groupby('date').mean()
df_eqCount= df_seismic.groupby('date').count()
df_eqCount['count'] = df_eqCount.datetime.astype('float64').values
df_eqMean['count'] = df_eqCount['count'].values
df_eqMean.head()

,Depth,MW,RMS,count
date,,,,
2015-05-01,1.194603,0.196984,0.045397,63.0
2015-05-02,1.246190,0.240714,0.049524,42.0
2015-05-03,0.995306,0.165510,0.040000,49.0
2015-05-04,1.247429,0.177143,0.047714,35.0
2015-05-05,1.202955,0.201364,0.057273,44.0
